## Buoyancy Lab

We'll start by redefining our error calculating toolkit.

In [80]:
import numpy as np
from typing import Iterable, Tuple, Union

# Define our rules...

def rule_1(c, error_val):
    return abs(c) * error_val

def rule_2(c, val, error_val, power):
    return abs(c * power * val ** (power - 1)) * error_val

def rule_3(*err_vals: Iterable[float]) -> float:
    """
    Calculate rule 4 from the paper 'Treatment of Data'.
    
    @param err_vals: A list of parameters, being the errors in each value. It is assumed they were summed 
                     together to get the final value.
    
    @returns: The error of all of the sum of the values...
    """
    total = 0
    
    for err in err_vals:
        total += err ** 2
    
    return np.sqrt(total)


def rule_4(value: float, *error_list: Iterable[Tuple[float, float, float]]) -> float:
    """
    Calculate rule 4 from the paper 'Treatment of Data'.
    
    @param value: The value of the thing we are trying to calculate the error of.
    @param error_list: A list of length 3 tuples. 
                       Each tuple should contain:
                       - A float: A value in the error formula.
                       - A float: The measured error in the above value.
                       - A float: The power of the above value in the multiplicative formula.

    @returns: A float, being the error in 'value'.
    """
    total = 0
    
    for x, x_err, power in error_list:
        total += (power * (x_err / x)) ** 2
        
    return abs(value) * np.sqrt(total)

# Adding this to the toolkit, checks if two values "agree" with each other.... Might add docs later...
FloatVec = Union[float, np.ndarray] 
BoolVec = Union[bool, np.ndarray]

def values_agree(val_1: FloatVec, err_1: FloatVec, val_2: FloatVec, err_2: FloatVec) -> BoolVec:
    r11, r12 = val_1 - err_1, val_1 + err_1
    r21, r22 = val_2 - err_2, val_2 + err_2
    
    r11 <= r21
    
    return ((r11 <= r21) & (r21 <= r12)) | ((r11 <= r22) & (r22 <= r12))

We'll also define a function for pretty printing our results.

In [51]:
def print_vals(vals, errs, units, msgs):
    for val, err, unit, msg in zip(vals, errs, units, msgs):
        print(f"{msg}: {val} ± {err} {unit}")
        
messages = [
    "Buoyancy Force for Section 1",
    "Buoyancy Force for Sections 1 & 2",
    "Buoyancy Force for Sections 1-3",
    "Buoyancy Force for Entire Cylinder"
]

units = ["N"] * 4

## Part 1 (Theoretical Buoyancy)

In [60]:
g = 9.8 # Accel due to gravity...

mass_water = (483.4 - 161.7) / 1000 # grams -> kg
mass_water_err = rule_3(0.1 / 1000, 0.1 / 1000)

volume_water = 326 / 1e6 # mL -> m^3
volume_water_err = 1 / 1e6 # mL -> m^3

water_d = mass_water / volume_water
water_d_err = rule_4(water_d, (mass_water, mass_water_err, 1), (volume_water, volume_water_err, -1))

heights = np.array([1.915, 3.83, 5.705, 7.72]) / 100 # cm -> m
heights_err = 0.05 / 1000 # mm -> m

r = (2.53 / 2) / 100 # cm -> m
r_err = (0.05 / 2) / 1000 # mm -> m

volume_cyl = np.pi * (r ** 2) * heights
volume_cyl_err = rule_4(volume_cyl, (r, r_err, 2), (heights, heights_err, 1))

buoyancy = volume_cyl * water_d * g
buoyancy_err = rule_4(boyancy, (volume_cyl, volume_cyl_err, 1), (water_d, water_d_err, 1))

print_vals(boyancy, boyancy_err, units, messages)

Buoyancy Force for Section 1: 0.09310204888474237 ± 0.0005270156809125469 N
Buoyancy Force for Sections 1 & 2: 0.18620409776948474 ± 0.0009662863339913572 N
Buoyancy Force for Sections 1-3: 0.2773614563380967 ± 0.001414098444625251 N
Buoyancy Force for Entire Cylinder: 0.37532523101316506 ± 0.0019006802208185595 N


## Part 2 (Experimental Buoyancy)

In [73]:
total_w = (111.3 / 1000) * 9.8 
total_w_err = (0.1 / 1000) * 9.8

apparent_w = (np.array([102.1, 91.9, 82.6, 72.6]) / 1000) * 9.8
apparent_w_err = (0.1 / 1000) * 9.8

buoyancy2 = total_w - apparent_w
buoyancy2_err = rule_3(total_w_err, apparent_w_err)

print_vals(boyancy2, [boyancy2_err] * len(boyancy2), units, messages)

print(values_agree(boyancy, boyancy_err, boyancy2, boyancy2_err))

Buoyancy Force for Section 1: 0.09016000000000002 ± 0.0013859292911256335 N
Buoyancy Force for Sections 1 & 2: 0.19011999999999984 ± 0.0013859292911256335 N
Buoyancy Force for Sections 1-3: 0.28126000000000007 ± 0.0013859292911256335 N
Buoyancy Force for Entire Cylinder: 0.37926000000000004 ± 0.0013859292911256335 N
[False False False False]
